In [3]:
import os

In [21]:
# Create the 5 fields of the COCO annotation format
info = {
    "description": "Chess board screenshot dataset",
    "version": "1.0",
    "year": 2024,
    "contributor": "Jackson Hall",
    "date_created": "2024-07-24",
}
licenses = []
images = []
annotations = []
categories = [
    {
        'id': 0,
        'name': 'chess board screenshot',
    }
]

In [36]:
import os

IMAGES_DIR = os.path.join('output', 'images')
BBOXES_DIR = os.path.join('output', 'bounding_boxes')

image_files = os.listdir(IMAGES_DIR)
bbox_files = os.listdir(BBOXES_DIR)

In [37]:
from PIL import Image
from tqdm.notebook import tqdm


images = []
annotations = []
with tqdm(total=len(image_files)) as pbar:
    for i, (image_file, bbox_file) in enumerate(zip(image_files, bbox_files)):
        # Read the image
        with open(os.path.join(IMAGES_DIR, image_file), 'rb') as f:
            image = Image.open(f)
        
        # Get the height/width of the image in pixels
        width, height = image.size
        image_id = i
        image = {
            "id": image_id,
            "height": height,
            "width": width,
            "file_name": os.path.join(IMAGES_DIR, image_file),
            "date_captured": "2024-07-25 00:00:00",
        }
        images.append(image)

        # Read the bounding box and add an annotation
        with open(os.path.join(BBOXES_DIR, bbox_file), 'r') as f:
            bboxes = f.read()

        x, y, w, h = map(int, bboxes.split(','))
        annotation = {
            'image_id': image_id,
            'category_id': 0,
            'bbox': [x, y, w, h]
        }
        annotations.append(annotation)
        pbar.update(1)


  0%|          | 0/2500 [00:00<?, ?it/s]

In [47]:
# Split the data into training and validation sets
TRAIN_VAL_RATIO = 0.8
train_size = int(len(image_files) * TRAIN_VAL_RATIO)
test_size = len(image_files) - train_size

train_images = images[:train_size]
train_annotations = annotations[:train_size]

val_images = images[train_size:]
val_annotations = annotations[train_size:]

train_dataset = {
    "info": info,
    "licenses": licenses,
    "images": train_images,
    "annotations": train_annotations,
    "categories": categories,
}
val_dataset = {
    "info": info,
    "licenses": licenses,
    "images": val_images,
    "annotations": val_annotations,
    "categories": categories,
}

In [48]:
import json

TRAIN_OUTPUT_FILE = 'output/chess_board_screenshots_train.coco.json'
VAL_OUTPUT_FILE = 'output/chess_board_screenshots_val.coco.json'

with open(TRAIN_OUTPUT_FILE, 'w') as f:
    json.dump(train_dataset, f)

with open(VAL_OUTPUT_FILE, 'w') as f:
    json.dump(val_dataset, f)

In [5]:
import os
import cv2


def convert_to_yolo_format(x, y, w, h, img_width, img_height):
    # Calculate the center of the box
    x_center = x + w / 2.0
    y_center = y + h / 2.0

    # Normalize the coordinates
    x_center /= img_width
    y_center /= img_height
    w /= img_width
    h /= img_height

    return x_center, y_center, w, h


def process_bounding_boxes(input_dir, output_dir, image_subdir, label_subdir):
    image_dir = os.path.join(input_dir, image_subdir)
    label_dir = os.path.join(input_dir, label_subdir)

    for phase in ["train", "val"]:
        image_phase_dir = os.path.join(image_dir, phase)
        label_phase_dir = os.path.join(label_dir, phase)
        output_label_phase_dir = os.path.join(output_dir, label_subdir, phase)

        if not os.path.exists(output_label_phase_dir):
            os.makedirs(output_label_phase_dir)

        for filename in os.listdir(label_phase_dir):
            if filename.endswith(".txt"):
                image_filename = filename.replace(".txt", ".jpg")
                image_path = os.path.join(image_phase_dir, image_filename)
                label_path = os.path.join(label_phase_dir, filename)

                # Read image to get its dimensions
                image = cv2.imread(image_path)
                img_height, img_width, _ = image.shape

                with open(label_path, "r") as file:
                    lines = file.readlines()

                yolo_lines = []
                for line in lines:
                    class_id, x, y, w, h = map(float, line.split())
                    x_center, y_center, norm_w, norm_h = convert_to_yolo_format(
                        x, y, w, h, img_width, img_height
                    )
                    yolo_line = (
                        f"{int(class_id)} {x_center} {y_center} {norm_w} {norm_h}\n"
                    )
                    yolo_lines.append(yolo_line)

                output_label_path = os.path.join(output_label_phase_dir, filename)
                with open(output_label_path, "w") as out_file:
                    out_file.writelines(yolo_lines)


# Define your directories
root_dir = r"C:\Users\jacks\OneDrive\Documents\Code\detect-chess-board-screenshots\synthetic_data_creation"
output_dir = os.path.join(root_dir, "output")
image_subdir = "images"
label_subdir = "labels"

# Process bounding boxes
process_bounding_boxes(root_dir, output_dir, image_subdir, label_subdir)

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'C:\\Users\\jacks\\OneDrive\\Documents\\Code\\detect-chess-board-screenshots\\synthetic_data_creation\\labels\\train'